## Exploratory Data Analysis

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
#set the plotting style
sns.set_style("whitegrid")

# Model preprocessing
from sklearn.preprocessing import StandardScaler

#Modelling
import statsmodels.formula.api as smf
import statsmodels.api as sm

# Model metrics and analysis
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from statsmodels.stats.anova import anova_lm

# Plotting tools
import plotly.offline as po
import plotly.graph_objs as pg

#Scikit for imputer tools
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/jamilditter/election_results/refs/heads/main/data/Voting_Breakdown_By_Socioeconomic_and_Demographic_Factors.csv')


InvalidURL: URL can't contain control characters. '/jamilditter/election_results/refs/heads/main/data/Voting Breakdown By Socioeconomic and Demographic Factors.csv' (found at least ' ')

In [ ]:
df["pop_18_24_per"] = df["Population 18-24"] / df["voting_age_pop"]
df["pop_25_44_per"] = df["Population 25-44"] / df["voting_age_pop"]
df["pop_45_64_per"] = df["Population 45-64"] / df["voting_age_pop"]
df["pop_65_per"] = df["Population 65+"] / df["voting_age_pop"]

In [ ]:
predictor_variables = ['unemployment_rate', 'median_income', 'med_age', 'bach_per', 'hs_per', 'voter_participation', 'pop_18_24_per', 'pop_25_44_per', 'pop_45_64_per', 'pop_65_per', 'voting_age_pop']

numerical_predictors = df[predictor_variables].select_dtypes(include='number').columns.to_list()

corr_matrix_dem = df[numerical_predictors+ ["dem_votes_per"]].corr()

plt.figure(figsize=(10,5))

sns.heatmap(corr_matrix_dem, vmax=1, vmin=-1, square=True, annot=True, cmap="viridis")

plt.tick_params(labelsize=12)

plt.show()

In [ ]:
corr_matrix_rep = df[numerical_predictors+ ["rep_votes_per"]].corr()

plt.figure(figsize=(10,5))

sns.heatmap(corr_matrix_rep, vmax=1, vmin=-1, square=True, annot=True, cmap="viridis")

plt.tick_params(labelsize=12)

plt.show()

In [ ]:
fig = sns.pairplot(
    data=df,
    vars=numerical_predictors + ['dem_votes_per'],
    kind="reg",
    plot_kws={"scatter_kws": {"alpha": 0.5, "color": "k", "s": 7},
    },
)

for ax in fig.axes.flat:
    ax.set_xlabel(ax.get_xlabel(), fontsize=8, rotation=30, ha='right')  # X-axis label size and rotation
    ax.set_ylabel(ax.get_ylabel(), fontsize=8)  # Y-axis label size

    # Rotate x-axis tick labels
    plt.setp(ax.get_xticklabels(), rotation=30, ha='right')

plt.show()

In [ ]:
fig = sns.pairplot(
    data=df,
    vars=numerical_predictors + ['rep_votes_per'],
    kind="reg",
    plot_kws={"scatter_kws": {"alpha": 0.5, "color": "k", "s": 7},
    },
)

for ax in fig.axes.flat:
    ax.set_xlabel(ax.get_xlabel(), fontsize=8, rotation=30, ha='right')  # X-axis label size and rotation
    ax.set_ylabel(ax.get_ylabel(), fontsize=8)  # Y-axis label size

    # Rotate x-axis tick labels
    plt.setp(ax.get_xticklabels(), rotation=30, ha='right')

plt.show()

In [ ]:
plt.figure(figsize=(10,3))

fractions = list(numerical_predictors)
fractions.remove('median_income')

sns.boxplot(data=df[fractions], color='k')

plt.ylabel('Proportion', fontsize=15)

plt.tick_params(labelsize=12)

plt.show()

In [ ]:
plt.figure(figsize=(3,3))

sns.boxplot(data=df, y = 'median_income', color='k')

plt.ylabel('Median Income ($)', fontsize=15)

plt.show()

## Modelling

### Single input models

In [ ]:
plt.figure(figsize=(6,6))

sns.regplot(data=df,
            x = 'median_income',
            y = 'average_act',
            color='blue',
            ci=False,
            scatter_kws={'color': 'black', 'edgecolors': 'white', 'linewidths': 1})

#Add axis labels
plt.xlabel('Median income ($)', fontsize=16)
plt.ylabel('Average ACT Score', fontsize=16)

#Increase the fontsize of the tick labels
plt.tick_params(labelsize=14)

plt.show()

In [ ]:
model_median_income = smf.ols(formula='average_act ~ median_income', data=df).fit()

In [ ]:
print(model_median_income.summary())

In [ ]:
y_hat = model_median_income.predict()

In [ ]:
np.sqrt(mean_squared_error(df['average_act'], y_hat)).round(3)

In [ ]:
mean_absolute_error(df['average_act'], y_hat)

In [ ]:
plt.figure(figsize=(6, 6))

plt.plot(y_hat, model_median_income.resid, 'ko', mec='w')
plt.axhline(0, color='r', linestyle='dashed', lw=2)

plt.xlabel('Predicted average ACT score', fontsize=16)
plt.ylabel('Residual average ACT score', fontsize=16)

plt.tick_params(labelsize=14)

plt.show()

In [ ]:
plt.figure(figsize=(6,6))

sns.regplot(data=df,
            x= 'median_income',
            y='average_act',
            color='blue',
            ci=False,
            scatter_kws={'color': 'black', 'edgecolors': 'white', 'linewidths': 1})

sns.regplot(data=df,
            x= 'median_income',
            y='average_act',
            order =2,
            color='orange',
            ci=False,
            scatter=False)

# Add axis labels
plt.xlabel('Median income ($)', fontsize=16)
plt.ylabel('Average ACT score', fontsize=16)

# Increase the fontsize of the tick labels
plt.tick_params(labelsize=14)

plt.show()

### Multiple Linear Regression Model

In [ ]:
model = smf.ols(
    formula='average_act ~ rate_unemployment + percent_college + percent_married + median_income +percent_lunch+Title_1',
    data=df).fit()

In [ ]:
print(model.summary())

In [ ]:
y_hat = model.predict()

In [ ]:
plt.figure(figsize=(5, 5))

plt.plot(y_hat, model.resid, 'ko', mec='w')
plt.axhline(0, color='r', linestyle='dashed', lw=2)

plt.xlabel('Predicted average ACT score', fontsize=16)
plt.ylabel('Residual average ACT score', fontsize=16)

plt.tick_params(labelsize=14)

plt.show()

In [ ]:
mean_absolute_error(df['average_act'], model.predict())

### Reduced Model

In [ ]:
model_reduced = smf.ols(formula='average_act ~ rate_unemployment + percent_lunch',data=df).fit()

In [ ]:
print(model_reduced.summary())

In [ ]:
plt.figure(figsize=(5, 5))

plt.plot(y_hat, model_reduced.resid, 'ko', mec='w')
plt.axhline(0, color='r', linestyle='dashed', lw=2)

plt.xlabel('Predicted average ACT score', fontsize=16)
plt.ylabel('Residual average ACT score', fontsize=16)

plt.tick_params(labelsize=14)

plt.show()

In [ ]:
mean_absolute_error(df['average_act'], model_reduced.predict())

In [ ]:
mae_full = mean_absolute_error(df['average_act'], model.predict())
mae_reduced = mean_absolute_error(df['average_act'], model_reduced.predict())

r2_full = model.rsquared
r2_reduced = model_reduced.rsquared

pd.DataFrame({'Mean Absolute Error': [mae_full, mae_reduced],
              'R-squared': [r2_full, r2_reduced]},
             index=['full model', 'reduced model']).round(4)

### Scaling

In [ ]:
predictor_variables = ['rate_unemployment', 'percent_college', 'percent_lunch']

In [ ]:
scaled_columns = [var + '_normalized' for var in predictor_variables]
print(scaled_columns)

In [ ]:
scaler = StandardScaler().fit(df[predictor_variables])

In [ ]:
df[scaled_columns] = scaler.transform(df[predictor_variables])

In [ ]:
df[scaled_columns].agg(['mean','std']).round(3)

In [ ]:
model_normalized = smf.ols(formula='average_act ~ rate_unemployment_normalized + percent_college_normalized + percent_lunch_normalized',data=df).fit()

In [ ]:
print(model_normalized.summary())

In [ ]:
mae_normalized = mean_absolute_error(df['average_act'], model_normalized.predict())
mae_reduced = mean_absolute_error(df['average_act'], model_reduced.predict())

r2_normalized = model_normalized.rsquared
r2_reduced = model_reduced.rsquared

pd.DataFrame({'Mean Absolute Error': [mae_normalized, mae_reduced],
              'R-squared': [r2_normalized, r2_reduced]},
             index=['normalized model', 'reduced model']).round(4)